In [33]:
import pandas as pd
from nltk import word_tokenize
from collections import Counter
import sys
import unittest

In [34]:
data = pd.read_csv("data/noticias_estadao_processado_4.csv")

### Sobre os dados

Os dados são referente a notícias, contendo 7643 observações e três variáveis, sendo elas referente a um identificador da notícia, título e conteúdo

In [35]:
data.shape

(7643, 3)

In [36]:
data.head()

,titulo,conteudo,idNoticia
0,11 dos eleitores do País são filiados a legendas,Há porém variações regionais nesse fenômeno En...,7617
1,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,412
2,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,415
3,13 de deputados do PMDB quer romper com PT,O Estado ouviu 54 dos 74 deputados do PMDB em ...,6736
4,2014 começou em 2007,O estudo do Estadão Dados publicado ontem sobr...,7611


### Agrupando título e conteúdo da notícia em uma mesma string

In [ ]:
data["noticia"] = data.apply(lambda row: (row["titulo"] + " " + row["conteudo"]).replace('.', '').lower(), axis=1)

### Gerando lista de tokens para cada texto

In [ ]:
tokens_by_rows = data.apply(lambda row : set(word_tokenize((row["noticia"].replace('.', '').lower()))), axis=1)

### Funções auxiliares

In [ ]:
def create_inverted_index(data, tokens_by_rows, id_column):
    '''
        Função que cria índice invertido
        Args:
            data (:obj: `DataFrame`): dataframe com dados das notícias
            tokens_by_rows (list): lista de sets com os tokens de cada notícia
    '''
    inverted_index = {}

    for i in range(len(tokens_by_rows)):
        for key in tokens_by_rows[i]:
            if key in inverted_index:
                inverted_index[key].append(data.at[i, id_column])
            else:
                inverted_index[key] = [data.at[i, id_column]]
    return inverted_index

In [ ]:
def get_token_less_docs(tokens, inverted_index):
    
    '''
        Função que recupera token com a menor lista de documentos
        Args:
            tokens (list): lista de tokens
            inverted_index (dict): dicionário do índice invertido
        Return:
            token_less_docs (str): token que possui o menor número de documentos
    '''
    less_docs_size = sys.maxsize
    token_less_docs = None
    for token in tokens:
        if len(inverted_index[token]) < less_docs_size:
            less_docs_size = len(inverted_index[token])
            token_less_docs = token
    return token_less_docs

In [ ]:
def get_documents_by_token(token, inverted_index):
    '''
        Recupera os documentos referentes a um token
        Args:
            token (str): string que representa um token
            inverted_index (dict): índice invertido
        Return
            resp (list): lista de documentos associados ao token
    '''
    try:
        resp = inverted_index[token]
    except:
        resp = []
    return resp

In [ ]:
def get_documents_by_tokens(text, operator,inverted_index):
    '''
        Recupera os ids de documentos entre dois tokens
        Args:
            text (str): string de busca com dois termos
            operator (str): operador lógico (AND / OR)
            inverted_index (dict): índice invertido
        Return:
            resp (list): lista com índices de documentos baseados nos tokens e operação lógica
            
    '''
    
    tokens = word_tokenize(text.lower())
    resp = []
    
    if operator == "and":
        
        token_less_docs = get_token_less_docs(tokens, inverted_index)
        tokens.remove(token_less_docs)
        
        smaller_list = get_documents_by_token(token_less_docs, inverted_index)
        lists_to_check = [get_documents_by_token(token, inverted_index) for token in tokens]
        
        for item in smaller_list:
            counter = 0
            for list_to_check in lists_to_check:
                if item not in list_to_check:
                    break
                else:
                    counter+=1
            if counter == len(lists_to_check):
                resp.append(item)       
    elif operator == "or":
        resp = list(set(sum([get_documents_by_token(token, inverted_index) for token in tokens], [])))
    
    return resp

In [ ]:
def search(str_search, inverted_index):
    '''
        Recupera os documentos para uma string de busca
        
        Args:
            str_search (str): String de busca
            inverted_index (dict): Dicionário de índice invertido.
    '''
    resp = []
    
    and_lists = str_search.split(" OR ")
    and_lists = [and_list.replace(" AND ", " ") for and_list in and_lists]
    
    for and_list in and_lists:
        resp.append(get_documents_by_tokens(and_list, "and", inverted_index))
    
    return list(set(sum(resp, [])))

### Criando o Índice Invertido

In [ ]:
inverted_index = create_inverted_index(data, tokens_by_rows, 'idNoticia')

## Exemplos para teste

### 1. debate, presidenciável (AND e OR)

In [ ]:
assert len(search("debate OR presidencial", inverted_index)) == 1770

In [ ]:
assert len(search("debate AND presidencial", inverted_index)) == 201

### 2. presidenciáveis, corruptos (AND e OR)

In [ ]:
assert len(search("presidenciáveis OR corruptos", inverted_index)) == 164

In [ ]:
assert len(search("presidenciáveis AND corruptos", inverted_index))== 0

### 3. Belo, Horizonte (AND e OR)

In [ ]:
assert len(search("Belo OR Horizonte", inverted_index)) == 331

In [ ]:
assert len(search("Belo AND Horizonte", inverted_index)) == 242

### 4. Campina, Grande (AND e OR)

In [ ]:
assert len(search("Campina OR Grande", inverted_index)) == len(inverted_index['campina']) + len(inverted_index['grande']) - len(search("Campina AND Grande", inverted_index))

In [ ]:
assert len(search("Campina OR Grande", inverted_index)) == 1656

In [ ]:
assert len(search("Campina AND Grande", inverted_index)) == 12

### EXTRAS

In [ ]:
assert len(search("Campina OR Grande OR Paraíba", inverted_index)) == 1719

Na teoria dos conjuntos, a propriedade lógica "OR" pode ser entendida como a operação de união. Um possível teste
para a entrada acima pode ser feito verificando a igualdade entre a cardinalidade de cada operação com os conjuntos
descritos abaixo:

$$ \mid A \cup B \cup C \mid  =  \mid A \mid  + \mid B \mid + \mid C \mid - \mid A \cap B \mid - \mid B \cap C \mid - \mid A \cap C \mid + \mid A \cap B \cap C \mid $$

In [ ]:
assert len(search("Campina OR Grande OR Paraíba", inverted_index)) == len(search("Campina", inverted_index)) + len(search("Grande", inverted_index)) + len(search("Paraíba", inverted_index)) - len(search("Campina AND Paraíba", inverted_index)) - len(search("Campina AND Grande", inverted_index)) - len(search("Grande AND Paraíba", inverted_index)) + len(search("Campina AND Grande AND Paraíba", inverted_index))

$$ \mid A \cap B \cap C \mid  = \mid A \cup B \cup C \mid  -  \mid A \mid  - \mid B \mid - \mid C \mid + \mid A \cap B \mid + \mid B \cap C \mid + \mid A \cap C \mid$$

In [ ]:
assert len(search("Campina AND Grande AND Paraíba", inverted_index)) == 8

In [ ]:
assert len(search("Campina AND Grande AND Paraíba", inverted_index)) == len(search("Campina OR Grande OR Paraíba", inverted_index)) - len(search("Campina", inverted_index)) - len(search("Grande", inverted_index)) - len(search("Paraíba", inverted_index)) + len(search("Campina AND Paraíba", inverted_index)) + len(search("Campina AND Grande", inverted_index)) + len(search("Grande AND Paraíba", inverted_index))